In [ ]:
# 1. Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydub

In [ ]:
# 3. 라이브러리 임포트
import os
import random
from pydub import AudioSegment
from openpyxl import Workbook

In [ ]:
def mix_audio_files(audio_files, output_path, silence_thresh=-50.0, target_length_ms=5000):
    """
    여러 오디오 파일을 믹싱하여 하나의 오디오 파일로 출력합니다.
    """
    first_audio = AudioSegment.from_file(audio_files[0])
    second_audio = AudioSegment.from_file(audio_files[1])
    mixed_audio = first_audio.overlay(second_audio)

    final_audio = mixed_audio[:target_length_ms]
    final_audio.export(output_path, format="wav")
    #print(f"{os.path.basename(output_path)} saved: mix {first_audio} and {second_audio}")

In [ ]:
import os
import random
from openpyxl import Workbook
import shutil

def create_mixed_audio_files(base_path, output_folder, num_files, spreadsheet_path="file_paths.xlsx"):
    """
    랜덤으로 선택된 오디오 파일을 믹싱하여 지정된 수만큼 파일을 생성하고,
    'train', 'test', 'valid' 폴더에 지정된 비율로 분류하여 저장합니다.
    """
    # 모든 폴더 필터링
    all_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    filtered_folders = [folder for folder in all_folders]

    # 출력 폴더 및 하위 폴더 생성
    subsets = ['train', 'test', 'valid']
    subset_counts = {'train': 400, 'test': 60, 'valid': 140}

    for subset in subsets:
        # os.makedirs(os.path.join(output_folder, subset), exist_ok=True)
        # os.makedirs(f'/content/drive/MyDrive/Dataset/Animal_Animal/s1/{subset}', exist_ok=True)
        # os.makedirs(f'/content/drive/MyDrive/Dataset/Animal_Animal/s2/{subset}', exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 's1'), exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 's2'), exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 'mixed_data'), exist_ok=True)

    # 스프레드시트 초기화
    wb = Workbook()
    ws = wb.active
    ws.append(["index", "first-file", "second-file", "subset"])  # 첫 번째 row 설정

    # 파일 생성 및 저장
    file_index = {'train': 0, 'test': 0, 'valid': 0}  # 각 subset의 생성된 파일 수
    for i in range(num_files):
        # 랜덤으로 폴더 두 개 선택
        random_folders = random.sample(filtered_folders, 2)

        # 각 폴더에서 파일 랜덤 선택
        selected_files = []
        selected_file_names = []  # 파일 이름 저장 리스트
        for folder in random_folders:
            folder_path = os.path.join(base_path, folder)
            all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
            if all_files:
                selected_file = random.choice(all_files)
                selected_files.append(os.path.join(folder_path, selected_file))
                selected_file_names.append(selected_file)  # 파일 이름 추가

        # 선택된 파일 이름 출력
        print(f"Animal_Animal_{i + 1}.wav saved: mix {selected_file_names}")

        # 파일 믹싱 및 저장
        if len(selected_files) == 2:  # 두 개 파일 모두 유효할 때만 믹싱
            # 현재 subset 결정
            for subset, count in subset_counts.items():
                if file_index[subset] < count:
                    current_subset = subset
                    break

            # Mixed 파일 저장
            subset_folder = os.path.join(output_folder, current_subset, 'mixed_data')
            output_path = os.path.join(subset_folder, f"Animal_Animal_{i + 1}.wav")
            mix_audio_files(selected_files, output_path=output_path)

            # 원본 파일 복사 및 저장
            # s1_path = f'/content/drive/MyDrive/Dataset/mk_1800)dataset/s1/{current_subset}/Animal_Animal_{i + 1}.wav'
            # s2_path = f'/content/drive/MyDrive/Dataset/Animal_Animal/s2/{current_subset}/Animal_Animal_{i + 1}.wav'
            s1_path = os.path.join(output_folder, current_subset, 's1', f"Animal_Animal_{i + 1}.wav")
            s2_path = os.path.join(output_folder, current_subset, 's2', f"Animal_Animal_{i + 1}.wav")
            shutil.copy(selected_files[0], s1_path)
            shutil.copy(selected_files[1], s2_path)

            # 한 줄씩 스프레드시트에 기록 (인덱스와 파일 경로, subset 정보)
            ws.append([i + 1, f"{random_folders[0]}/{selected_file_names[0]}",
                       f"{random_folders[1]}/{selected_file_names[1]}", current_subset])

            # 해당 subset의 파일 개수 증가
            file_index[current_subset] += 1

    # 엑셀 저장
    # wb.save(spreadsheet_path)
    # print(f"Spreadsheet saved at {spreadsheet_path}")
    return wb

In [ ]:
base_path = '/content/drive/MyDrive/Dataset/RawData/animal'  # 기본 경로
output_folder = '/content/drive/MyDrive/Dataset/mk_1800_dataset/'  # 출력 폴더
num_files = 1200  # 생성할 믹싱 파일 수 #16000

wb = create_mixed_audio_files(base_path, output_folder, num_files=num_files)

Animal_Animal_1.wav saved: mix ['9.참새_163.wav', 'trimmed_S-210808_A_204_D_004_0001.mp3의 사본']
Animal_Animal_2.wav saved: mix ['Cat_8.wav', 'trimmed_S-210808_A_204_D_013_0001.mp3의 사본']
Animal_Animal_3.wav saved: mix ['6.까치_8.wav', 'Monkey_2 (1).wav']
Animal_Animal_4.wav saved: mix ['Elephant_4.wav', 'Lion_12.wav']
Animal_Animal_5.wav saved: mix ['Elephant_4.wav', '5.멧돼지_399.wav']
Animal_Animal_6.wav saved: mix ['9.참새_335.wav', 'Cat_109.wav']
Animal_Animal_7.wav saved: mix ['Cat_34.wav', 'Monkey_8.wav']
Animal_Animal_8.wav saved: mix ['Bird_10.wav', '8.소쩍새_20.wav']
Animal_Animal_9.wav saved: mix ['Monkey_9.wav', 'Dog_35.wav']
Animal_Animal_10.wav saved: mix ['trimmed_S-211023_A_102_C_075_0001.mp3의 사본', 'trimmed_S-210724_A_204_D_002_0001.mp3의 사본']
Animal_Animal_11.wav saved: mix ['Aslan_3.wav', 'trimmed_S-211026_A_103_D_332_0001.mp3의 사본']
Animal_Animal_12.wav saved: mix ['9.참새_420.wav', 'Cat_96.wav']
Animal_Animal_13.wav saved: mix ['trimmed_S-210724_